# G7 Python 入门课程
## 项目1 Python操作入门

入门可能主要是读取税前薪水和可能的纳税额，根据正确的公式，判断是否正确

**提示**：这样的文字将会指导你如何使用 iPython Notebook 来完成项目。

In [2]:
# 检查你的Python版本
from sys import version_info
if version_info.major != 2 and version_info.minor != 7:
    raise Exception('请使用Python 2.7来完成此项目')

In [3]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# 数据可视化代码
from titanic_visualizations import survival_stats
from IPython.display import display
%matplotlib inline

# 加载数据集
in_file = 'data.csv'
full_data = pd.read_csv(in_file)

# 显示数据列表中的数据
display(full_data)

,name,salary,tax_maybe
0,wang,2500,0
1,zhang,7000,105
2,li,8000,205
3,song,9000,405
4,tang,50000,800


# 数据样本中，我们可以看到的特征

- **name**：名称
- **salary**：税前薪水
- **tax_maybe**：可能的缴税额度

个税的计算方式
![xxx](https://img-blog.csdn.net/20171017113915227?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvVG9nZXRoZXJfQ1o=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)

In [4]:
def limit_scope(data, data_min, data_max):
    """返回数据的取值区间"""
    if data < data_min:
        return data_min
    elif data > data_max:
        return data_max
    else:
        return data

In [5]:
def salary_tax(salary):
    """返回用户缴纳三险一金后的应计税工资"""

    endowment_insurance_rate = 0.08  # 养老保险费率
    endowment_insurance_min = 2193  # 养老保险基数最小值
    endowment_insurance_max = 16445  # 养老保险基数最大值

    provident_rate = 0.12  # 公积金费率
    provident_max = 20972  # 公积金基数最大值
    provident_min = 1500  # 公积金基数最小值

    hospital_rate = 0.02  # 医疗保险费率
    losejob_rate = 0.01  # 失业保险费率

    # 计算应扣的养老保险费用
    endowment_insurance = limit_scope(salary, endowment_insurance_min, endowment_insurance_max) \
                          * endowment_insurance_rate

    # 计算应扣公积金
    provident = limit_scope(salary, provident_min, provident_max) * provident_rate

    # 计算应扣的医保费用
    hospital = salary * hospital_rate

    # 计算应扣的失业保险费用
    lose_job = salary * losejob_rate

    # 计算扣除三险一金后的剩余工资
    return salary - endowment_insurance - provident - hospital - lose_job

In [6]:
def tax_calculator(salary):
    """返回用户应缴纳的个税"""

    tax_free_point = 3500
    # tax_level = [1500, 4500, 9000, 35000, 55000, 80000, 80001]
    # tax_rate = [3, 10, 20, 25, 30, 35, 40]
    # tax_deducted_amount = [0, 105, 555, 1005, 2755, 5505, 13505]

    tax_level = [
        (1500, 3, 0),
        (4500,10,105),
        (9000,20,555),
        (35000,25,1005),
        (55000,30,2755),
        (80000,35,5505),
        (9e+99,40,13505)
    ]

    # 计算用户缴纳三险一金后的应计税工资
    res_money = salary_tax(salary) - tax_free_point

    # 计算用户所在的缴税级别
    if res_money <= 0:
        return 0
    else:
        for boundary, rate, deduction in tax_level:
            if res_money < boundary:
                return round(res_money * rate / 100.0 - deduction, 2)

In [7]:
def float_equal(num1, num2):
    """比较两个浮点数是否相同"""
    
    if abs(num1 - num2) < 0.01:
        return True

In [22]:
taxs = {}

for index, staff in full_data.iterrows():
    if staff['salary'] <= 0:
        error_message = staff['name'] + "工资出错啦！！！"
        taxs[staff['name']] = None
    else:
        tax = tax_calculator(staff['salary'])
        res_salary = salary_tax(staff['salary']) - tax
        print  staff['name'] + "税前工资：" + str(staff['salary']) + "，税后工资：" + str(res_salary)
        
        taxs[staff['name']] = tax

        # 检查所得税预估值是否正确
        if float_equal(tax, staff['tax_maybe']):
            print staff['name'] + "的应扣税为：" + str(tax) + ", 与预估值一致，so cool！\n"
        else:
            print staff['name'] + "的应扣税为：" + str(tax) + ", 预估值出错啦！\n"
            

wang税前工资：2500，税后工资：1925.0
wang的应扣税为：0, 与预估值一致，so cool！

zhang税前工资：7000，税后工资：5306.0
zhang的应扣税为：84.0, 预估值出错啦！

li税前工资：8000，税后工资：5999.0
li的应扣税为：161.0, 预估值出错啦！

song税前工资：9000，税后工资：6692.0
song的应扣税为：238.0, 预估值出错啦！

tang税前工资：50000，税后工资：35072.43
tang的应扣税为：9595.33, 预估值出错啦！



## 扩展内容
将正确的纳税额导出到export.csv，并计算预测的正确率


In [26]:
def export() :
    """导出正确的纳税额到export.csv, tax作为最后一列"""
    
    in_file = 'data.csv'
    full_data = pd.read_csv(in_file)
    out_file = 'export.csv'

    data_taxs = DataFrame({'name': taxs.keys(), 'tax_actual': taxs.values()})
    full_data = pd.merge(full_data, data_taxs, how='left', on='name')
    #print full_data
    
    
    full_data.to_csv(out_file, encoding="utf-8", index=False)
    print "export done"
    
export()

# 加载数据集
in_file = 'export.csv'
export_data = pd.read_csv(in_file)

# 显示数据列表中的前几项数据
display(export_data.head())

export done


,name,salary,tax_maybe,tax_actual
0,wang,2500,0,0.00
1,zhang,7000,105,84.00
2,li,8000,205,161.00
3,song,9000,405,238.00
4,tang,50000,800,9595.33


In [27]:
def accuracy_score():
    # 计算正确率
    socre = 0
    right_times = 0
    wrong_times = 0
    
    full_data = pd.read_csv('export.csv')
    for index, staff in full_data.iterrows():
        if float_equal(staff['tax_maybe'], staff['tax_actual']):
            right_times += 1
        else:
            wrong_times += 1
            
        socre = float(right_times) / (right_times + wrong_times)
        
    return "accuracy of {:.2f}%.".format(socre * 100)
    
accuracy_score()

'accuracy of 20.00%.'

> **注意**: 当你写完了所有**4个TODO**。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)** 把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。